In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-26 04:31:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-08-26 04:31:14 (16.9 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create a mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[stoi[c] for c in s]
decode1 = lambda l : ([itos[i] for i in l])
decode = lambda l : ''.join([itos[i] for i in l])


print(encode('shubhankar mondal'))
print(decode1(encode('shubhankar mondal')))
print(decode(encode('shubhankar mondal')))
print(len(decode(encode('shubhankar mondal'))))

[57, 46, 59, 40, 46, 39, 52, 49, 39, 56, 1, 51, 53, 52, 42, 39, 50]
['s', 'h', 'u', 'b', 'h', 'a', 'n', 'k', 'a', 'r', ' ', 'm', 'o', 'n', 'd', 'a', 'l']
shubhankar mondal
17


In [1]:
import torch # type: ignore
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)

2.4.0+cu121
True
12.1


In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text) , dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])
print(data[:1000].shape)

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
train_data [0:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context} the target is {target}')

When input is tensor([18]) the target is 47
When input is tensor([18, 47]) the target is 56
When input is tensor([18, 47, 56]) the target is 57
When input is tensor([18, 47, 56, 57]) the target is 58
When input is tensor([18, 47, 56, 57, 58]) the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [12]:
torch.manual_seed(1237)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [13]:
xb, yb = get_batch('train')
print('############  INPUTS  #############')
print(xb.shape)
print(xb)
print('###########  TARGET  #############')
print(yb.shape)
print(yb)

print('--------------------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f'When input is {context.tolist()} the target is: {target}')

############  INPUTS  #############
torch.Size([4, 8])
tensor([[25, 17, 30, 31, 17, 32, 10,  0],
        [50,  1, 51, 43, 58,  6,  1, 45],
        [ 0, 23, 21, 26, 19,  1, 30, 21],
        [53, 52, 42,  1, 35, 39, 58, 41]])
###########  TARGET  #############
torch.Size([4, 8])
tensor([[17, 30, 31, 17, 32, 10,  0, 21],
        [ 1, 51, 43, 58,  6,  1, 45, 43],
        [23, 21, 26, 19,  1, 30, 21, 15],
        [52, 42,  1, 35, 39, 58, 41, 46]])
--------------------
When input is [25] the target is: 17
When input is [25, 17] the target is: 30
When input is [25, 17, 30] the target is: 31
When input is [25, 17, 30, 31] the target is: 17
When input is [25, 17, 30, 31, 17] the target is: 32
When input is [25, 17, 30, 31, 17, 32] the target is: 10
When input is [25, 17, 30, 31, 17, 32, 10] the target is: 0
When input is [25, 17, 30, 31, 17, 32, 10, 0] the target is: 21
When input is [50] the target is: 1
When input is [50, 1] the target is: 51
When input is [50, 1, 51] the target is: 43
When i

In [14]:
print(xb)

tensor([[25, 17, 30, 31, 17, 32, 10,  0],
        [50,  1, 51, 43, 58,  6,  1, 45],
        [ 0, 23, 21, 26, 19,  1, 30, 21],
        [53, 52, 42,  1, 35, 39, 58, 41]])


In [15]:
import torch.nn as nn # type: ignore
from torch.nn import functional as F # type: ignore
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])


tensor(4.7251, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [16]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [17]:
batch_size = 32
for steps in range(1000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.7218432426452637


In [18]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=2000)[0].tolist()))


olylvLLko'TMyatyIoconxad.?-tNSqYPsx&bF.oiR;BD$dZBMZv'K f bRSmIKptRPly:AUC&$zLK,qUEy&Ay;ZxjKVhmrdagC-bTop-QJe.H?x
JGF&pwst-P sti.hlEsu;w:w a BG:tLhMk,epdhlay'sVzLq--ERwXUzDnq-bn czXxxI&V&Pynnl,s,Ioto!uvixwC-IJXElrgm C-.bcoCPJ
IMphsevhO AL!-K:AIkpre,
rPHEJUzV;P?uN3b?ohoRiBUENoV3B&jumNL;Aik,
xf -IEKROn JSyYWW?n 'ay;:weO'AqVzPyoiBL? seAX3Dot,iy.xyIcf r!!ul-Koi:x pZrAQly'v'a;vEzN
BwowKo'MBqF$PPFb
CjYX3beT,lZ qdda!wfgmJP
DUfNXmnQU mvcv?nlnQF$JUAAywNocd  bGSPyAlprNeQnq-GRSVUP.Ja!IBoDqfI&xJM AXEHV&DKvRSVJuFphSPUGqULI'MeDnXzLIcky,VzIf:MDtRF$HYDqmIJuUEXCKR;hoTR:PPm,egmbbHEXrhaU'xx;
pjJU!jTo--wNlsXeBwnolByI'naPYZTToTI'Vvvn CEX:ELgFGSAJM!U-
YulxhSPlbvEDUALTXEJUCXLtgq-ZuBAXKFCUALMomyEQORotiHoCIsBVDKoEuC&sMa!hceV;AD&Q!Bv'MHewo;3QWW:CUNCOMk.iBOkze!LKHnsV&QuoitelHARJERjZArdfB' ErBBHR:pbQ!LOJAOpCjRpBLTj$;&MOW:x;I--tV.Q!zecoTo'QJ;JOd.:kOKIiyansivviOKpw siy; 'tYGbo,&tiIIjsGXJ$gsz'JNDq,q-
Y phipa!UA.yN3kH-Wo'w-pxlc3jly' f ERLObyuDZYIgqFlaymNOXK:CIIyhZxcK:VKir 
Id,q-aphhJFhZAJjm; ?OWIVjThiv'jgw!
'WDq-mreZ

Mathematical tricks for self attention

In [20]:
torch.manual_seed(1337)
B, T, C = 4,8,4   #Batch, Time, Channel
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 4])

In [23]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]   #(T*C)
        xbow[b,t] = torch.mean(xprev, dim=0)
x[0],xbow[0]

(tensor([[ 0.1808, -0.0700, -0.3596, -0.9152],
         [ 0.6258,  0.0255,  0.9545,  0.0643],
         [ 0.3612,  1.1679, -1.3499, -0.5102],
         [ 0.2360, -0.2398, -0.9211,  1.5433],
         [ 1.3488, -0.1396,  0.2858,  0.9651],
         [-2.0371,  0.4931,  1.4870,  0.5910],
         [ 0.1260, -1.5627, -1.1601, -0.3348],
         [ 0.4478, -0.8016,  1.5236,  2.5086]]),
 tensor([[ 0.1808, -0.0700, -0.3596, -0.9152],
         [ 0.4033, -0.0222,  0.2974, -0.4254],
         [ 0.3892,  0.3745, -0.2517, -0.4537],
         [ 0.3509,  0.2209, -0.4190,  0.0456],
         [ 0.5505,  0.1488, -0.2781,  0.2295],
         [ 0.1192,  0.2062,  0.0161,  0.2897],
         [ 0.1202, -0.0465, -0.1519,  0.2005],
         [ 0.1611, -0.1409,  0.0575,  0.4890]]))

In [30]:
a = torch.ones(4,4)
a = torch.tril(a)
a = a/torch.sum(a,-1, keepdim=True)
b = torch.randint(0,10,(4,3)).float()
c = a @ b
print(a)
print('____________')
print(b)
print('____________')
print(c)


tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])
____________
tensor([[5., 4., 6.],
        [1., 1., 5.],
        [1., 4., 9.],
        [2., 5., 0.]])
____________
tensor([[5.0000, 4.0000, 6.0000],
        [3.0000, 2.5000, 5.5000],
        [2.3333, 3.0000, 6.6667],
        [2.2500, 3.5000, 5.0000]])


In [43]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(dim=1,keepdim=True)
xbow2 = wei @ x # (B,T,T)* (B,T,C)--->(B,T,C)
wei, x[0], xbow[0], xbow2[0]
torch.allclose(xbow, xbow2)

True

In [45]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril==0 , float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [46]:
wei = F.softmax(wei, dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [47]:
xbow3 = wei @ x
x[0],xbow2[0],xbow3[0]

(tensor([[ 0.1808, -0.0700, -0.3596, -0.9152],
         [ 0.6258,  0.0255,  0.9545,  0.0643],
         [ 0.3612,  1.1679, -1.3499, -0.5102],
         [ 0.2360, -0.2398, -0.9211,  1.5433],
         [ 1.3488, -0.1396,  0.2858,  0.9651],
         [-2.0371,  0.4931,  1.4870,  0.5910],
         [ 0.1260, -1.5627, -1.1601, -0.3348],
         [ 0.4478, -0.8016,  1.5236,  2.5086]]),
 tensor([[ 0.1808, -0.0700, -0.3596, -0.9152],
         [ 0.4033, -0.0222,  0.2974, -0.4254],
         [ 0.3892,  0.3745, -0.2517, -0.4537],
         [ 0.3509,  0.2209, -0.4190,  0.0456],
         [ 0.5505,  0.1488, -0.2781,  0.2295],
         [ 0.1192,  0.2062,  0.0161,  0.2897],
         [ 0.1202, -0.0465, -0.1519,  0.2005],
         [ 0.1611, -0.1409,  0.0575,  0.4890]]),
 tensor([[ 0.1808, -0.0700, -0.3596, -0.9152],
         [ 0.4033, -0.0222,  0.2974, -0.4254],
         [ 0.3892,  0.3745, -0.2517, -0.4537],
         [ 0.3509,  0.2209, -0.4190,  0.0456],
         [ 0.5505,  0.1488, -0.2781,  0.2295],
         

In [48]:
torch.allclose(xbow,xbow3)

True

In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4, 8, 32
x = torch.randn(B,T,C)
# lets see a single head perform self attention
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)

k = key(x)      #(B,T,16)
q = query(x)    #(B,T,16)
wei = q @ k.transpose(-2,-1)    #(B,T,16)@(B,16,T)-->(B,T,T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0,float('-inf'))
wei = F.softmax(wei,dim=-1)

v = value(x)        #(B,T,16)
out = wei @ v

In [5]:
out.shape

torch.Size([4, 8, 16])

In [6]:
out[0]

tensor([[-0.1571,  0.8801,  0.1615, -0.7824, -0.1429,  0.7468,  0.1007, -0.5239,
         -0.8873,  0.1907,  0.1762, -0.5943, -0.4812, -0.4860,  0.2862,  0.5710],
        [ 0.6764, -0.5477, -0.2478,  0.3143, -0.1280, -0.2952, -0.4296, -0.1089,
         -0.0493,  0.7268,  0.7130, -0.1164,  0.3266,  0.3431, -0.0710,  1.2716],
        [ 0.4823, -0.1069, -0.4055,  0.1770,  0.1581, -0.1697,  0.0162,  0.0215,
         -0.2490, -0.3773,  0.2787,  0.1629, -0.2895, -0.0676, -0.1416,  1.2194],
        [ 0.1971,  0.2856, -0.1303, -0.2655,  0.0668,  0.1954,  0.0281, -0.2451,
         -0.4647,  0.0693,  0.1528, -0.2032, -0.2479, -0.1621,  0.1947,  0.7678],
        [ 0.2510,  0.7346,  0.5939,  0.2516,  0.2606,  0.7582,  0.5595,  0.3539,
         -0.5934, -1.0807, -0.3111, -0.2781, -0.9054,  0.1318, -0.1382,  0.6371],
        [ 0.3428,  0.4960,  0.4725,  0.3028,  0.1844,  0.5814,  0.3824,  0.2952,
         -0.4897, -0.7705, -0.1172, -0.2541, -0.6892,  0.1979, -0.1513,  0.7666],
        [ 0.1866, -0.0

In [7]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [8]:
q[0]

tensor([[-0.6567,  0.0283,  0.0094, -0.6995, -0.3604,  0.8376, -0.4446,  0.1228,
          0.6276, -0.6222,  0.3483,  0.2411,  0.5409, -0.2605,  0.3612, -0.0436],
        [-0.3932,  0.8220, -0.7027,  0.0954, -0.1222, -0.1518, -0.5024, -0.4636,
          0.1176,  1.4282, -0.5812,  0.1401,  0.9604,  0.0410, -0.6214, -0.6347],
        [ 0.2157, -0.3507,  0.0022,  0.4232, -0.2284, -0.0732, -0.3412,  0.9647,
         -0.5178,  0.0921, -0.5043,  0.8388,  0.6149, -0.0109, -0.5569,  0.5820],
        [ 0.9000, -0.1272,  0.5458,  0.4254, -0.4513, -0.0212,  0.1711,  0.2599,
         -0.9978,  0.4890,  0.1737, -0.0700, -0.3113,  0.3748, -0.1848, -0.6379],
        [ 0.0332,  0.5886, -0.4437,  0.3775, -0.6826, -0.2775,  0.4673, -1.2956,
          0.6603,  0.1633, -1.7573, -0.6582, -0.2302, -0.0862, -0.0060,  0.7573],
        [ 0.2098,  0.0439, -0.0702,  0.0727, -0.2012, -1.7539,  1.0369,  0.1163,
          0.2956,  0.3231,  0.5052,  0.7011, -0.2844, -0.7844,  0.4782, -0.5170],
        [ 0.6100, -0.3

In [11]:
q = torch.randn(B,T,C)
k = torch.randn(B,T,C)
wei = q @k.transpose(-2,-1)
print(q.var())
print(k.var())
print(wei.var())

tensor(1.0185)
tensor(1.0148)
tensor(28.4248)


In [12]:
wei = wei * head_size**-0.5
print(wei.var())

tensor(1.7765)


In [13]:
torch.softmax(torch.tensor([-0.1, -0.2, 0.2 , 0.5, 0.3]),dim = -1)

tensor([0.1522, 0.1378, 0.2055, 0.2774, 0.2271])

In [15]:
torch.softmax(torch.tensor([-0.1, -0.2, 0.2 , 0.5, 0.3])*5,dim = -1)

tensor([0.0298, 0.0181, 0.1335, 0.5984, 0.2202])